# CS354N
---
**Name: Somya Mehta**

**Roll No.: 190001058**

**Assignment - 9**



In [ ]:
import numpy as np
import random



# ================================================================ #
def shape(x):
  return np.array(x).shape

def collectData(f):
  res = []
  for line in f:
    x = line.split(',')
    y = []
    for item in x:
      y.append(float(item.strip()))
    res.append(y)
  return np.array(res)

def initializeParameters(inputFeatures, neuronsInHiddenLayers, outputFeatures):
  W1 = np.random.randn(neuronsInHiddenLayers, inputFeatures) * 0.01
  W2 = np.random.randn(outputFeatures, neuronsInHiddenLayers) * 0.01
  b1 = np.zeros((neuronsInHiddenLayers, 1))
  b2 = np.zeros((outputFeatures, 1))

  parameters = {"W1" : W1, "b1": b1,
        "W2" : W2, "b2": b2}
  return parameters

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
		# compute the derivative of the sigmoid function ASSUMING
		# that x has already been passed through the 'sigmoid'
		# function
		return x * (1 - x)

def predict(X_test, params):
  W1, W2, b1, b2 = params['W1'], params['W2'], params['b1'], params['b2']
  A2 = []
  for X in X_test:
    X = X.reshape(-1, 1)
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2.append(sigmoid(Z2))
    
  return np.array(A2).reshape(-1, 1)

def accuracy_score(y_pred, y_true):
  c, n = 0.0, 0.0
  for y, yt in zip(y_pred, y_true):
    if y == yt: c += 1
    n += 1
  return c / n 

def forward_prop(X, params):
    W1, W2, b1, b2 = params['W1'], params['W2'], params['b1'], params['b2']
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    # here the cache is the data of previous iteration
    # This will be used for backpropagation
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    return A2, cache
  
# Here Y is actual output
def compute_cost(A2, Y):
    # implementing the mean squared error
    diff = Y - A2
    diff = diff * diff
    cost_sum = np.sum(diff)  
    cost = 1/2 * cost_sum
    cost = np.squeeze(cost)
    return cost

def back_propagate(X, Y, params, cache, learning_rate):
  W1, W2, b1, b2 = params['W1'], params['W2'], params['b1'], params['b2']
  # Retrieve also A1 and A2 from dictionary "cache"
  A1 = cache['A1']
  A2 = cache['A2']

  # Backward propagation: calculate dW1, db1, dW2, db2.
  # layer 1 from out
  
  dA2 = A2 - Y
  dZ2 = dA2 * sigmoid_deriv(A2) # element wise multiplication
  dW2 = np.dot(dZ2, A1.T)
  db2 = np.sum(dZ2, axis = 1, keepdims = True)
  # layer 2

  dA1 = np.dot(W2.T, dZ2)
  dZ1 = dA1 * sigmoid_deriv(A1)
  dW1 = np.dot(dZ1, np.array(X).T)
  db1 = np.sum(dZ1, axis = 1, keepdims = True)

  # Updating the parameters according to algorithm
  W1 = W1 - learning_rate * dW1
  b1 = b1 - learning_rate * db1
  W2 = W2 - learning_rate * dW2
  b2 = b2 - learning_rate * db2

  return W1, W2, b1, b2



# Generate 7 input majority
def generateData(n):

  x = np.random.randint(0, 2, size=(n, 7))
  y = []
  for i in range(len(x)):
    c = 0
    for item in x[i]:
      if item == 1: c += 1
    if c >= 4:
      y.append(1)
    else:
      y.append(0)
  y = np.array(y)
  return x, y



  
  
  

In [ ]:
# encode labels
X_train = np.zeros((20, 7))
y_train = np.zeros((20, 1))
X_test = np.zeros((10, 7))
y_test = np.zeros((10, 1))

X_train, y_train = generateData(len(X_train))
X_test, y_test = generateData(len(X_test))
# shape of data X_train (753, 64), X_test (363, 64), y_train (753, 1), y_test (363, 1)
# training the model


input_dims = len(X_train[0])
output_dims = 1
hidden_layer_dims = 15
params = initializeParameters(input_dims, hidden_layer_dims, output_dims)
# out, cache = forward_prop(X_train[0], params['W1'], params['W2'],params['b1'], params['b2'])
# cost = compute_cost(out, y_train[0])
# print(cost)

num_iterations = 200
learning_rate = 0.01

for i in range(0, num_iterations):
  for X, y in zip(X_train, y_train):
    X = X.reshape(-1, 1)
    y = y.reshape(-1, 1)
    A2, cache = forward_prop(X, params)
    cost = compute_cost(A2, y)
    W1, W2, b1, b2 = back_propagate(X, y, params, cache, learning_rate)
    params['W1'], params['W2'], params['b1'], params['b2'] = W1, W2, b1, b2
  # Print the cost every 1000 iterations
  if i % 25 == 0 or i == 199:
      print ("Cost after iteration % i: % f" % (i, cost))

# testing
threshold = 0.5
y_pred = predict(X_test, params)
for i in range(0, y_pred.shape[0]):
  if y_pred[i] >= threshold: y_pred[i] = 1
  else: y_pred[i] = 0
acc = accuracy_score(y_pred, y_test)  
print(params['W1'])
print(params['W2'])

print(f"accuracy is {acc}")

Cost after iteration  0:  0.129505
Cost after iteration  25:  0.168801
Cost after iteration  50:  0.178597
Cost after iteration  75:  0.181103
Cost after iteration  100:  0.181796
Cost after iteration  125:  0.182033
Cost after iteration  150:  0.182152
Cost after iteration  175:  0.182242
Cost after iteration  199:  0.182320
[[-1.08341564e-03 -1.96908860e-03 -2.56837502e-02 -1.96968487e-02
  -1.63404116e-02 -2.15603504e-02 -8.42483303e-06]
 [-1.35627806e-03  3.39537979e-03 -2.10430229e-02 -1.94188293e-02
  -7.60192066e-03 -6.56017560e-04 -3.28145905e-03]
 [-5.77621234e-03 -1.54106850e-03 -2.19959763e-02  1.32391341e-02
  -1.09876724e-02 -1.28070038e-02  3.96899672e-05]
 [-8.80705796e-03  7.26460782e-03 -7.24272897e-03 -2.20961597e-02
  -2.96337264e-04 -2.08730889e-02 -4.70493248e-03]
 [ 3.54239195e-03 -4.67662671e-03 -1.60199287e-02  5.57666736e-03
   6.70351153e-03  1.67216405e-04 -1.86756271e-02]
 [-1.35981751e-02 -1.52742139e-03 -1.12320915e-02  3.25519973e-03
   1.01404719e-03 -5.

In [ ]:
#Q2

# ================================================================ #
def shape(x):
  return np.array(x).shape

def collectData(f):
  res = []
  for line in f:
    x = line.split(',')
    y = []
    for item in x:
      y.append(float(item.strip()))
    res.append(y)
  return np.array(res)

def initializeParameters(inputFeatures, neuronsInHiddenLayers, outputFeatures):
  W1 = np.random.randn(neuronsInHiddenLayers, inputFeatures) * 0.01
  W2 = np.random.randn(outputFeatures, neuronsInHiddenLayers) * 0.01
  b1 = np.zeros((neuronsInHiddenLayers, 1))
  b2 = np.zeros((outputFeatures, 1))

  parameters = {"W1" : W1, "b1": b1,
        "W2" : W2, "b2": b2}
  return parameters

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
		# compute the derivative of the sigmoid function ASSUMING
		# that x has already been passed through the 'sigmoid'
		# function
		return x * (1 - x)

def predict(X_test, params):
  W1, W2, b1, b2 = params['W1'], params['W2'], params['b1'], params['b2']
  A2 = []
  for X in X_test:
    X = X.reshape(-1, 1)
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2.append(sigmoid(Z2))
    
  return np.array(A2).reshape(-1, 1)

def accuracy(predictions, targets):
  return 0.5 * np.sum((predictions - targets) ** 2)

def forward_prop(X, params):
    W1, W2, b1, b2 = params['W1'], params['W2'], params['b1'], params['b2']
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    # here the cache is the data of previous iteration
    # This will be used for backpropagation
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    return A2, cache
  
# Here Y is actual output
def compute_cost(A2, Y):
    # implementing the mean squared error
    diff = Y - A2
    diff = diff * diff
    cost_sum = np.sum(diff)  
    cost = 1/2 * cost_sum
    cost = np.squeeze(cost)
    return cost

def back_propagate(X, Y, params, cache, learning_rate):
  W1, W2, b1, b2 = params['W1'], params['W2'], params['b1'], params['b2']
  # Retrieve also A1 and A2 from dictionary "cache"
  A1 = cache['A1']
  A2 = cache['A2']

  # Backward propagation: calculate dW1, db1, dW2, db2.
  # layer 1 from out
  
  dA2 = A2 - Y
  dZ2 = dA2 * sigmoid_deriv(A2) # element wise multiplication
  dW2 = np.dot(dZ2, A1.T)
  db2 = np.sum(dZ2, axis = 1, keepdims = True)
  # layer 2

  dA1 = np.dot(W2.T, dZ2)
  dZ1 = dA1 * sigmoid_deriv(A1)
  dW1 = np.dot(dZ1, np.array(X).T)
  db1 = np.sum(dZ1, axis = 1, keepdims = True)

  # Updating the parameters according to algorithm
  W1 = W1 - learning_rate * dW1
  b1 = b1 - learning_rate * db1
  W2 = W2 - learning_rate * dW2
  b2 = b2 - learning_rate * db2

  return W1, W2, b1, b2

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

# Generate 7 input majority
def input_1(n = 100):
  Train = np.zeros(n)
  Label = np.zeros(n)
  for i in range(n):
    Train[i] = i+1
    Label[i] = 1 / Train[i]
  X, y = unison_shuffled_copies(Train, Label)
  len_train = int(0.7*n)
  # len_test = int(0.3*n)
  return X[0:len_train], y[0:len_train], X[len_train:], y[len_train:]

  
  
# encode labels
X_train, y_train, X_test, y_test = input_1()
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

input_dims = len(X_train[0])
output_dims = 1
print ("enter number of neurons in hidden layer: ")
hidden_layer_dims = int(input())
params = initializeParameters(input_dims, hidden_layer_dims, output_dims)

num_iterations = 1000
learning_rate = 0.01

for i in range(0, num_iterations):
  for X, y in zip(X_train, y_train):
    X = X.reshape(-1, 1)
    y = y.reshape(-1, 1)
    A2, cache = forward_prop(X, params)
    cost = compute_cost(A2, y)
    W1, W2, b1, b2 = back_propagate(X, y, params, cache, learning_rate)
    params['W1'], params['W2'], params['b1'], params['b2'] = W1, W2, b1, b2
  # Print the cost every 1000 iterations
  if i % 100 == 0:
      print ("Cost after iteration % i: % f" % (i, cost))

# testing
y_pred = predict(X_test, params)
loss = accuracy(y_pred, y_test)  

print(f"Loss is {loss}")

enter number of neurons in hidden layer: 
5
Cost after iteration  0:  0.088319
Cost after iteration  100:  0.000617
Cost after iteration  200:  0.000483
Cost after iteration  300:  0.000468
Cost after iteration  400:  0.000461
Cost after iteration  500:  0.000456
Cost after iteration  600:  0.000449
Cost after iteration  700:  0.000440
Cost after iteration  800:  0.000430
Cost after iteration  900:  0.000416
Loss is 0.01640504788073724


In [ ]:
#Q3

# ================================================================ #
class NeuralNetwork:
  def __init__(self, layers, alpha=0.1):
    self.W = []
    self.layers = layers
    self.alpha = alpha
    for i in np.arange(0, len(layers) - 2):
      w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
      self.W.append(w / np.sqrt(layers[i]))
    w = np.random.randn(layers[-2] + 1, layers[-1])
    self.W.append(w / np.sqrt(layers[-2]))
  
  def __repr__(self):
    return "NeuralNetwork: {}".format(
      "-".join(str(l) for l in self.layers))
  
  def sigmoid(self, x):
    return 1.0 / (1 + np.exp(-x))

  def sigmoid_deriv(self, x):
    return x * (1 - x)

  def fit(self, X, y, epochs=1000, displayUpdate=100):
    X = np.c_[X, np.ones((X.shape[0]))]
    for epoch in np.arange(0, epochs):
      for (x, target) in zip(X, y):
        self.fit_partial(x, target)
      if epoch == 0 or (epoch + 1) % displayUpdate == 0:
        loss = self.calculate_loss(X, y)
        print("[INFO] epoch={}, loss={:.7f}".format(
          epoch + 1, loss))
      	
  def fit_partial(self, x, y):
    A = [np.atleast_2d(x)]
    for layer in np.arange(0, len(self.W)):
      net = A[layer].dot(self.W[layer])
      out = self.sigmoid(net)
      A.append(out)
    error = A[-1] - y
    D = [error * self.sigmoid_deriv(A[-1])]
    for layer in np.arange(len(A) - 2, 0, -1):
      delta = D[-1].dot(self.W[layer].T)
      delta = delta * self.sigmoid_deriv(A[layer])
      D.append(delta)
    D = D[::-1]
    for layer in np.arange(0, len(self.W)):
      self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])
   
  def predict(self, X, addBias=True):
    p = np.atleast_2d(X)
    if addBias:
      p = np.c_[p, np.ones((p.shape[0]))]
    for layer in np.arange(0, len(self.W)):
      p = self.sigmoid(np.dot(p, self.W[layer]))
    return p

  def calculate_loss(self, X, targets):
    targets = np.array(targets)
    predictions = self.predict(X, addBias=False)
    predictions = predictions.reshape(-1)
    loss = 0.5 * np.sum((predictions - targets) ** 2)
    return loss
  
def collectData(f):
  label, input = [], []
  res = []
  for line in f:
    x = line.split(',')
    y = []
    for item in x:
      y.append((item.strip()))
    res.append(y)
  
  for i in range(1, len(res)):
      line = res[i]
      x = []
      y = 0
      for item in line:
        if(item == 'setosa'):
          y = 0
        elif(item == 'versicolor'):
          y = 1
        else:
          x.append(float(item))
      label.append(y)
      input.append(x)
  
  return np.array(input), np.array(label)

X_train, y_train = [], []
X_test, y_test = [], []
with open('iris_train.csv', 'r') as f:
  X_train, y_train = collectData(f)
with open('iris_test.csv', 'r') as f:
  X_test, y_test = collectData(f)


X = X_train
y = y_train

nn = NeuralNetwork([4, 10, 10, 1], alpha=0.001)
nn.fit(X, y, epochs=1000)

X = X_test
y = y_test

true_cases = 0
all_cases = 0

for (x, target) in zip(X, y):
  pred = nn.predict(x)[0][0]
  step = 1 if pred > 0.5 else 0
  all_cases += 1
  if(target == step):
    true_cases+=1
    
acc = true_cases / all_cases
print("[Final] acc = {}".format(acc))

[INFO] epoch=1, loss=11.0864545
[INFO] epoch=100, loss=9.8823195
[INFO] epoch=200, loss=9.7020409
[INFO] epoch=300, loss=9.5471500
[INFO] epoch=400, loss=9.3553272
[INFO] epoch=500, loss=9.1141828
[INFO] epoch=600, loss=8.7940903
[INFO] epoch=700, loss=8.3327822
[INFO] epoch=800, loss=7.6668143
[INFO] epoch=900, loss=6.8529715
[INFO] epoch=1000, loss=5.9792535
[Final] acc = 1.0
